In [ ]:
from imports import *


# Import konfigurasi dari file config.py
from config.config_aio import AIO_SERVER, AIO_USERNAME, AIO_KEY, AIO_FEED

In [ ]:
# # Inisialisasi Firebase
# cred = credentials.Certificate('firebase_sdk_admin_led_switch.json')
# firebase_admin.initialize_app(cred, {
#     'databaseURL': 'https://led-switch-6c690-default-rtdb.firebaseio.com'  # Ganti dengan URL Realtime Database Anda
# })

In [ ]:
app = Flask(__name__)

# Adafruit IO Configuration

# Gunakan konfigurasi dalam kode Anda
print("Server:", AIO_SERVER)
print("Username:", AIO_USERNAME)
print("Feed:", AIO_FEED)

# Folder untuk menyimpan foto yang diterima
UPLOAD_FOLDER = './uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

model = load_model("assets/drowsiness_model.h5")  # grayscale

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

EYE_CLOSED_THRESHOLD = 3  
FRAME_COUNT = 0  
last_eye_status = "open" 
start_time = None  

In [ ]:
def preprocess_image(image):
    # 1. Konversi ke grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 2. Menghilangkan noise menggunakan Gaussian Blur
    gray_blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # 3. Menajamkan gambar
    kernel_sharpening = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    gray_sharpened = cv2.filter2D(gray_blurred, -1, kernel_sharpening)

    # 4. Deteksi wajah
    faces = face_cascade.detectMultiScale(gray_sharpened, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        return None  # Tidak ada wajah terdeteksi

    # 5. Ambil wajah pertama yang terdeteksi
    x, y, w, h = faces[0]
    face = gray_sharpened[y:y+h, x:x+w]

    # 6. Alignment wajah
    eyes = face_cascade.detectMultiScale(face)
    if len(eyes) >= 2:  # Pastikan ada minimal dua mata terdeteksi
        # Ambil dua mata pertama
        eye1 = eyes[0]
        eye2 = eyes[1]
        
        # Pastikan eye1 adalah mata kiri dan eye2 mata kanan
        if eye1[0] > eye2[0]:
            eye1, eye2 = eye2, eye1
        
        # Koordinat pusat mata
        eye1_center = (eye1[0] + eye1[2] // 2, eye1[1] + eye1[3] // 2)
        eye2_center = (eye2[0] + eye2[2] // 2, eye2[1] + eye2[3] // 2)

        # Sudut rotasi untuk horizontal alignment
        dx = eye2_center[0] - eye1_center[0]
        dy = eye2_center[1] - eye1_center[1]
        angle = np.degrees(np.arctan2(dy, dx))

        # Rotasi wajah agar horizontal
        M = cv2.getRotationMatrix2D(eye1_center, angle, 1)
        face = cv2.warpAffine(face, M, (face.shape[1], face.shape[0]))

    # 7. Ekstraksi bagian wajah (mata dan mulut)
    face_height, face_width = face.shape
    top_cut = int(face_height * 0.3)  # Hapus dahi
    bottom_cut = int(face_height * 0.8)  # Fokus pada mata dan mulut
    face_cropped = face[top_cut:bottom_cut, :]  # Ekstraksi area wajah yang relevan

    # 8. Resize ke dimensi standar
    face_resized = cv2.resize(face_cropped, (64, 64))

    # 9. Normalisasi pixel
    face_normalized = face_resized / 255.0

    # 10. Tambahkan dimensi untuk kompatibilitas dengan model
    face_final = np.expand_dims(face_normalized, axis=-1)
    face_final = np.expand_dims(face_final, axis=0)

    return face_final


In [ ]:
# Callback for MQTT connection
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT broker")
        client.subscribe(AIO_FEED)
    else:
        print(f"Failed to connect, return code {rc}")

In [ ]:
# Callback for MQTT message
def on_message(client, userdata, msg):
    print(f"Received message from MQTT feed: {msg.topic}, Payload size: {len(msg.payload)} bytes")

    try:
        # Jika data dikirim dalam bentuk base64, Anda perlu mendekode
        img_data = base64.b64decode(msg.payload)  # Dekode base64 (jika itu yang digunakan)

        # Mengubah bytes menjadi gambar menggunakan PIL
        image = Image.open(io.BytesIO(img_data))
        
        # Mengubah gambar menjadi format yang bisa diterima oleh OpenCV
        image = np.array(image)

        # Pastikan gambar sudah dalam format yang benar
        if len(image.shape) == 3 and image.shape[2] == 3:  # Memastikan gambar berwarna RGB
            print("Image decoded successfully")

        else:
            print("Invalid image format received")  

    except Exception as e:
        print(f"Error: Failed to decode image from payload. Exception: {str(e)}")

    # Save the image locally for debugging
    cv2.imwrite(os.path.join(UPLOAD_FOLDER, "mqtt_image.jpg"), image)

    # Preprocess and predict
    processed_image = preprocess_image(image)
    if processed_image is None:
        print("Error: No face detected in the image.")
        return

    prediction = model.predict(processed_image)
    eye_status = "closed" if prediction[0][0] > 0.5 else "open"
    print(f"Eye status: {eye_status}")



In [ ]:
# # kode untuk penggunaan buffer

# # Daftar untuk menyimpan gambar yang diterima
# image_buffer = []

# # Callback untuk menerima pesan MQTT
# def on_message(client, userdata, msg):
#     print(f"Received message from MQTT feed: {msg.topic}, Payload size: {len(msg.payload)} bytes")

#     try:
#         # Jika data dikirim dalam bentuk base64, Anda perlu mendekode
#         img_data = base64.b64decode(msg.payload)  # Dekode base64 (jika itu yang digunakan)

#         # Mengubah bytes menjadi gambar menggunakan PIL
#         image = Image.open(io.BytesIO(img_data))
        
#         # Mengubah gambar menjadi format yang bisa diterima oleh OpenCV
#         image = np.array(image)

#         # Pastikan gambar sudah dalam format yang benar
#         if len(image.shape) == 3 and image.shape[2] == 3:  # Memastikan gambar berwarna RGB
#             print("Image decoded successfully")
#         else:
#             print("Invalid image format received")  

#     except Exception as e:
#         print(f"Error: Failed to decode image from payload. Exception: {str(e)}")
#         return

#     # Menyimpan gambar ke dalam buffer untuk analisis lebih lanjut
#     image_buffer.append(image)

#     # Jika sudah ada 3 gambar dalam buffer, lakukan analisis
#     if len(image_buffer) >= 3:
#         # Analisis status pengemudi berdasarkan 3 gambar
#         eye_status_count = {"open": 0, "closed": 0}
#         yawn_status_count = {"open": 0, "closed": 0}

#         # Proses ketiga gambar
#         for img in image_buffer:
#             processed_image = preprocess_image(img)
#             if processed_image is None:
#                 print("Error: No face detected in the image.")
#                 continue

#             prediction = model.predict(processed_image)
#             eye_status = "closed" if prediction[0][0] > 0.5 else "open"
#             print(f"Eye status: {eye_status}")

#             # Cek status mulut (yawn detection) jika model mendeteksi mulut terbuka
#             if is_yawning(processed_image):  # Anda bisa menambahkan fungsi deteksi mulut terbuka di sini
#                 yawn_status = "open"  # Mulut terbuka menandakan menguap
#             else:
#                 yawn_status = "closed"  # Mulut tertutup menandakan tidak menguap
            
#             # Hitung frekuensi status mata dan mulut
#             if eye_status == "open":
#                 eye_status_count["open"] += 1
#             else:
#                 eye_status_count["closed"] += 1

#             if yawn_status == "open":
#                 yawn_status_count["open"] += 1
#             else:
#                 yawn_status_count["closed"] += 1

#         # Tentukan status pengemudi berdasarkan analisis tiga gambar
#         if eye_status_count["closed"] == 3:
#             print("Pengemudi mengantuk - Mata tertutup di ketiga gambar.")
#             send_sleep_status("mengantuk")
#         elif yawn_status_count["open"] == 3:
#             print("Pengemudi menguap - Mulut terbuka di ketiga gambar.")
#             send_sleep_status("menguap")
#         elif eye_status_count["closed"] > 0 and yawn_status_count["open"] > 0:
#             print("Pengemudi segar - Tidak semua gambar menunjukkan mata tertutup/mulut terbuka.")
#             send_sleep_status("segar")
#         elif eye_status_count["open"] == 3:
#             print("Pengemudi tidak mengantuk - Mata terbuka di ketiga gambar.")
#             send_sleep_status("tidak mengantuk")

#         # Reset buffer untuk gambar berikutnya
#         image_buffer.clear()

# # Fungsi untuk mengirim status ke sistem (misalnya menggunakan MQTT atau API)
# def send_sleep_status(status):
#     print(f"Sending status: {status}")
#     # Kode untuk mengirim status (misalnya melalui MQTT atau API)
#     # mqtt_client.publish("your_topic", status)

# # Fungsi tambahan untuk deteksi mulut terbuka (yawn)
# def is_yawning(image):
#     # Implementasikan deteksi mulut terbuka pada gambar (gunakan model atau algoritma lainnya)
#     # Ini adalah contoh fungsi kosong, Anda dapat menyesuaikannya dengan kebutuhan Anda.
#     return False  # Ganti dengan implementasi sebenarnya.

In [ ]:
# MQTT Client Setup
mqtt_client = Client()
mqtt_client.username_pw_set(AIO_USERNAME, AIO_KEY)
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

mqtt_client.connect(AIO_SERVER, 1883)
mqtt_client.loop_start()

@app.route('/')
def index():
    return "Server is running and connected to MQTT feed."

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
# def process_frame(current_eye_status):

#     global FRAME_COUNT, last_eye_status, start_time

#     if current_eye_status == "closed":
#         if last_eye_status == "closed":
#             FRAME_COUNT += 3
#         else:
#             FRAME_COUNT = 3
#             start_time = time.time() 
#     else:
#         FRAME_COUNT = 0
#         start_time = None 

#     last_eye_status = current_eye_status

#     if FRAME_COUNT >= EYE_CLOSED_THRESHOLD and (time.time() - start_time >= 3.5):
#         return "sleepy"
#     else:
#         return "not sleepy"

In [ ]:
# @app.route('/process_image', methods=['POST'])
# def process_image():
#     global FRAME_COUNT, last_eye_status, start_time

#     # Mengambil file gambar yang diupload
#     if 'image' not in request.files:
#         return jsonify({"error": "No image file provided"}), 400

#     file = request.files['image']
#     image = Image.open(io.BytesIO(file.read()))

#     image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

#     processed_image = preprocess_image(image)

#     if processed_image is None:
#         return jsonify({"error": "No face detected in the image"}), 400

#     # Misalkan Anda menggunakan model yang sudah dilatih
#     prediction = model.predict(processed_image)

#     if prediction[0][0] > 0.5:
#         current_eye_status = "closed"
#     else:
#         current_eye_status = "open"

#     result = process_frame(current_eye_status)

#     return jsonify({"result": result})

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)

# @app.route('/')
# def index():
#     return "Server is running and ready to accept images."

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', port=5000)  # Jalankan server di port 5000